# RLHF: Reinforcement Learning from Human Feedback

## O que é RLHF?

RLHF (Reinforcement Learning from Human Feedback) é uma técnica de fine-tuning que permite alinhar modelos de linguagem com preferências humanas. Ao invés de treinar apenas com dados estáticos, o modelo aprende a partir de feedback humano iterativo, similar a como ChatGPT foi treinado.

## Por que RLHF é importante?

Modelos de linguagem pré-treinados (como GPT-2, GPT-3) são treinados para prever o próximo token com base em dados da internet. Isso significa que eles aprendem padrões estatísticos, mas não necessariamente aprendem a gerar respostas **úteis**, **seguras** ou **alinhadas** com valores humanos.

RLHF resolve isso através de três etapas:

1. **Supervised Fine-tuning (SFT)**: Treina o modelo com exemplos de alta qualidade
2. **Reward Modeling**: Treina um modelo para prever preferências humanas
3. **Reinforcement Learning**: Otimiza o modelo para maximizar o reward

## Componentes deste Notebook:

### 1. Modelo Generativo (Policy)
- **Base**: GPT-2 pré-treinado
- **Função**: Gerar completions de texto
- **Treinamento**: Otimizado via gradientes do reward model

### 2. Modelo de Recompensa (Reward Model)
- **Base**: BERT pré-treinado
- **Função**: Aprender a prever preferências humanas
- **Input**: Par (prompt, completion)
- **Output**: Score entre 0 (ruim) e 1 (bom)

### 3. Modelo de Referência
- **Cópia do modelo original**: Usado para calcular KL divergence
- **Função**: Prevenir que o modelo se desvie muito do comportamento original
- **Frozen**: Não é atualizado durante o treinamento

## Pipeline RLHF Implementado:

```
1. Geração → 2. Feedback Humano → 3. Treina Reward Model → 4. Otimiza Policy
```

### Objetivo deste Notebook:
Treinar GPT-2 para gerar continuações do prompt **"Donald Trump is a"** que sejam avaliadas positivamente por humanos, demonstrando como RLHF pode influenciar o comportamento do modelo.

## 1. Importações de Bibliotecas

Importamos as bibliotecas necessárias para o treinamento RLHF.

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BertTokenizer,
    BertModel
)
import numpy as np
from tqdm.auto import tqdm
import warnings
import copy
warnings.filterwarnings('ignore')

## 2. Configuração do Dispositivo

### Aceleradores de Hardware

O treinamento de modelos de linguagem requer grande capacidade computacional. Este código detecta automaticamente o melhor acelerador disponível:

#### 1. **MPS (Metal Performance Shaders)**
- Hardware: Apple Silicon (M1, M2, M3, etc.)
- Vantagem: Acelera operações em GPUs da Apple
- Performance: 5-10x mais rápido que CPU

#### 2. **CUDA**
- Hardware: GPUs NVIDIA
- Vantagem: Suporte nativo do PyTorch, altamente otimizado
- Performance: 10-100x mais rápido que CPU (depende da GPU)

#### 3. **CPU**
- Hardware: Processador convencional
- Vantagem: Funciona em qualquer máquina
- Desvantagem: Muito mais lento (não recomendado para produção)

### Por que o Device é Importante?

Operações de deep learning envolvem milhões de multiplicações de matrizes. Aceleradores como GPUs executam essas operações em paralelo, enquanto CPUs processam sequencialmente.

In [2]:
if torch.backends.mps.is_available():
    device = torch.device("mps")
    print("✅ Usando MPS (Apple Silicon)")
elif torch.cuda.is_available():
    device = torch.device("cuda")
    print("✅ Usando CUDA")
else:
    device = torch.device("cpu")
    print("⚠️ Usando CPU")

print(f"Device: {device}")

✅ Usando MPS (Apple Silicon)
Device: mps


## 3. Modelo de Recompensa (BERTRewardModel)

### O que é um Reward Model?

Em RLHF, não podemos usar loss functions tradicionais (como cross-entropy) porque não temos labels de "resposta correta". Em vez disso, temos **preferências humanas**: humanos podem dizer qual de duas respostas é melhor.

O Reward Model é treinado para **imitar essas preferências**, aprendendo a atribuir scores altos para boas respostas e scores baixos para respostas ruins.

### Arquitetura do BERTRewardModel

```
Input: "[prompt] [SEP] [completion]"
    ↓
BERT Encoder (contexto)
    ↓
[CLS] Token Embedding (768 dims)
    ↓
Dropout (0.1)
    ↓
Linear (768 → 256)
    ↓
ReLU Activation
    ↓
Dropout (0.1)
    ↓
Linear (256 → 1)
    ↓
Sigmoid (output entre 0-1)
    ↓
Reward Score
```

### Por que BERT?

**BERT** (Bidirectional Encoder Representations from Transformers) é ideal para reward modeling porque:

1. **Bidirectional**: Processa o texto completo (prompt + completion) em contexto
2. **Pre-trained**: Já entende semântica e sintaxe da língua inglesa
3. **[CLS] Token**: Representa o significado global da sequência inteira

### Componentes da Arquitetura:

#### 1. BERT Base
- **Parâmetros**: ~110M
- **Camadas**: 12 transformer layers
- **Hidden Size**: 768 dimensões
- **Função**: Extrair representações contextuais

#### 2. Dropout (0.1)
- **Função**: Regularização para prevenir overfitting
- **Mecanismo**: Desativa aleatoriamente 10% dos neurônios durante treinamento

#### 3. Reward Head (MLP)
- **Camada 1**: 768 → 256 (compressão de features)
- **ReLU**: Não-linearidade (permite aprender padrões complexos)
- **Camada 2**: 256 → 1 (agregação para score único)
- **Sigmoid**: Normaliza output para intervalo [0, 1]

### Interpretação do Score:
- **1.0**: Resposta excelente (alinhada com preferências)
- **0.5**: Resposta neutra ou mediana
- **0.0**: Resposta ruim (não alinhada)

### Freeze BERT?

O parâmetro `freeze_bert` permite:
- **False** (padrão): Fine-tuna BERT inteiro (melhor performance, mais lento)
- **True**: Congela BERT, treina apenas reward head (mais rápido, menos flexível)

In [3]:
class BERTRewardModel(nn.Module):
    """
    Modelo de recompensa baseado em BERT que aprende com feedback humano.
    Prediz um score de qualidade para cada resposta gerada.
    """
    def __init__(self, model_name='bert-base-uncased', freeze_bert=False):
        super().__init__()
        self.bert = BertModel.from_pretrained(model_name)
        self.dropout = nn.Dropout(0.1)
        
        # Camada de recompensa: prediz score de 0 (ruim) a 1 (bom)
        self.reward_head = nn.Sequential(
            nn.Linear(self.bert.config.hidden_size, 256),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(256, 1),
            nn.Sigmoid()  # Output entre 0 e 1
        )
        
        if freeze_bert:
            for param in self.bert.parameters():
                param.requires_grad = False
    
    def forward(self, input_ids, attention_mask=None):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        cls_output = outputs.last_hidden_state[:, 0, :]  # [CLS] token
        cls_output = self.dropout(cls_output)
        reward_score = self.reward_head(cls_output)
        return reward_score

## 4. Carregamento de Modelos e Tokenizers

### Modelos Utilizados

#### 1. GPT-2 (Generative Pre-trained Transformer 2)

**Policy Model** - O modelo que será otimizado:
- **Parâmetros**: ~124M (versão base)
- **Arquitetura**: Decoder-only transformer (12 layers)
- **Pré-treinamento**: 40GB de texto da internet
- **Função**: Gerar continuações de texto autoregressivamente

**Por que GPT-2?**
- Open-source e amplamente documentado
- Tamanho gerenciável para experimentos
- Boa performance em geração de texto

#### 2. Modelo de Referência (Reference Model)

```python
ref_model = copy.deepcopy(generation_model)
```

**Função crítica no RLHF:**
- É uma **cópia congelada** do modelo original
- **Não é atualizado** durante o treinamento
- Usado para calcular **KL Divergence**

**Por que precisamos dele?**

Sem o modelo de referência, o RLHF pode sofrer de **reward hacking**:
- O modelo aprende a "enganar" o reward model
- Gera respostas que maximizam reward de forma artificial
- Pode produzir texto nonsense mas com alto score

A **KL penalty** mantém o modelo próximo do comportamento original:

```
KL(Policy || Reference) = divergência entre distribuições de probabilidade
```

Se a policy se afasta muito da referência, a penalidade aumenta.

#### 3. BERT (Bidirectional Encoder Representations from Transformers)

**Base do Reward Model:**
- **Parâmetros**: ~110M
- **Arquitetura**: Encoder-only transformer
- **Pré-treinamento**: Masked Language Modeling + Next Sentence Prediction
- **Função**: Entender contexto bidirecionalmente

### Tokenizers

#### AutoTokenizer (GPT-2)
- **Método**: Byte-Pair Encoding (BPE)
- **Vocab Size**: 50,257 tokens
- **Padding**: Configurado como EOS token

#### BertTokenizer
- **Método**: WordPiece
- **Vocab Size**: 30,522 tokens
- **Tokens Especiais**: [CLS], [SEP], [PAD], [MASK]

### Otimizadores: AdamW

**AdamW** (Adam with Weight Decay) é uma versão melhorada do Adam:

```python
# Reward Model: lr=2e-5
reward_optimizer = torch.optim.AdamW(reward_model.parameters(), lr=2e-5, weight_decay=0.01)

# Policy Model: lr=1e-5 (mais conservador)
policy_optimizer = torch.optim.AdamW(generation_model.parameters(), lr=1e-5, weight_decay=0.01)
```

**Configurações:**
- **Learning Rate (Reward)**: 2e-5 - Taxa de aprendizado moderada
- **Learning Rate (Policy)**: 1e-5 - Mais conservador (modelo generativo é sensível)
- **Weight Decay**: 0.01 - Regularização L2 para prevenir overfitting

**Por que LRs diferentes?**
- Reward model: Treina com supervised learning (mais estável)
- Policy model: Treina com RL (requer mais cuidado para não divergir)

In [4]:
# Carregar modelo generativo e tokenizer (Inglês)
gen_model_name = 'gpt2'
gen_tokenizer = AutoTokenizer.from_pretrained(gen_model_name)
gen_tokenizer.pad_token = gen_tokenizer.eos_token
generation_model = AutoModelForCausalLM.from_pretrained(gen_model_name).to(device)
ref_model = copy.deepcopy(generation_model).to(device)

# Carregar tokenizer e modelo para reward (BERT Inglês)
bert_model_name = 'bert-base-uncased'
bert_tokenizer = BertTokenizer.from_pretrained(bert_model_name)

# Modelo de recompensa
reward_model = BERTRewardModel(model_name=bert_model_name, freeze_bert=False).to(device)

# Otimizadores
reward_optimizer = torch.optim.AdamW(reward_model.parameters(), lr=2e-5, weight_decay=0.01)
policy_optimizer = torch.optim.AdamW(generation_model.parameters(), lr=1e-5, weight_decay=0.01)

print("✅ Modelos inicializados")
print(f"📊 Parâmetros treináveis (reward): {sum(p.numel() for p in reward_model.parameters() if p.requires_grad):,}")
print(f"📊 Parâmetros treináveis (policy): {sum(p.numel() for p in generation_model.parameters() if p.requires_grad):,}")

✅ Modelos inicializados
📊 Parâmetros treináveis (reward): 109,679,361
📊 Parâmetros treináveis (policy): 124,439,808


## 5. Função para Geração de Completions

### Geração Autoregressiva

Modelos de linguagem como GPT-2 geram texto **token por token**, onde cada novo token é predito baseado nos tokens anteriores:

```
Input: "Donald Trump is a"
Step 1: Prediz próximo token → "businessman"
Step 2: Input: "Donald Trump is a businessman" → Prediz "and"
Step 3: Input: "Donald Trump is a businessman and" → Prediz "former"
...
```

### Parâmetros de Geração

#### 1. **max_new_tokens=10**
Limita o número de tokens gerados. Para demonstração, usamos apenas 10 tokens por completion.

#### 2. **do_sample=True**
Ativa **sampling estocástico** em vez de greedy decoding:
- **Greedy**: Sempre escolhe o token mais provável (determinístico, repetitivo)
- **Sampling**: Amostra da distribuição de probabilidade (mais diverso)

#### 3. **top_p=0.95 (Nucleus Sampling)**

Também chamado de **nucleus sampling**, seleciona tokens do menor conjunto cuja probabilidade acumulada seja ≥ p.

**Exemplo:**
```
Tokens disponíveis:
- "president": 40%
- "businessman": 30%
- "politician": 20%
- "leader": 8%
- "person": 2%

top_p=0.95:
Acumula: 40% + 30% + 20% = 90% (ainda < 95%)
Adiciona: "leader" (8%) = 98% ≥ 95% ✓
Samplea de: {president, businessman, politician, leader}
```

**Vantagens:**
- Evita tokens extremamente improváveis
- Mantém diversidade entre tokens plausíveis
- Adapta-se automaticamente à distribuição (flat vs peaked)

#### 4. **temperature=0.7**

Controla a "aleatoriedade" da distribuição:

```python
logits_adjusted = logits / temperature
```

**Efeitos:**
- **temp = 1.0**: Distribuição original (não modificada)
- **temp < 1.0** (ex: 0.7): Mais focado (menos aleatório)
  - Aumenta probabilidade dos tokens mais prováveis
  - Reduz probabilidade dos menos prováveis
- **temp > 1.0**: Mais uniforme (mais aleatório)

**Exemplo prático:**
```
Original (temp=1.0):
- "president": 40%
- "businessman": 30%
- "politician": 20%

Com temp=0.7 (mais focado):
- "president": 48%
- "businessman": 32%
- "politician": 15%

Com temp=1.5 (mais aleatório):
- "president": 35%
- "businessman": 31%
- "politician": 23%
```

### Fluxo da Função:

1. **Tokenização**: Converte prompt para input_ids
2. **Geração**: Model.generate() produz novos tokens
3. **Decodificação**: Converte tokens de volta para texto
4. **Extração**: Remove o prompt original, retorna apenas completion

In [5]:
def generate_completion(model, tokenizer, prompt, max_new_tokens=10):
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    outputs = model.generate(
        inputs['input_ids'],
        attention_mask=inputs['attention_mask'],
        max_new_tokens=max_new_tokens,
        do_sample=True,
        top_p=0.95,
        temperature=1.0,
        pad_token_id=tokenizer.pad_token_id
    )
    full_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    completion = full_text[len(prompt):].strip()
    return completion

## 6. Sistema de Coleta de Feedback Humano

### Por que Feedback Humano?

Em tarefas como geração de texto, não existe uma única "resposta correta". Diferentes completions podem ser:
- Factualmente corretas mas ofensivas
- Gramaticalmente perfeitas mas irrelevantes
- Criativas mas inadequadas

Humanos possuem **juízo de valor** e **conhecimento contextual** que métricas automáticas não capturam.

### Processo de Coleta

#### 1. Apresentação
```
💬 Prompt: Donald Trump is a
🤖 Resposta: Donald Trump is a businessman and former president
```

#### 2. Avaliação Humana
O usuário classifica a resposta em 3 níveis:
- **1 (Bom)**: Resposta apropriada, factual, útil
- **2 (Neutro)**: Resposta aceitável mas não ideal
- **3 (Ruim)**: Resposta inadequada, ofensiva ou sem sentido

#### 3. Conversão para Reward
```python
1 → 1.0  (reward máximo)
2 → 0.5  (reward neutro)
3 → 0.0  (reward mínimo)
```

Esta normalização para [0, 1] facilita o treinamento com sigmoid no reward model.

### Classe HumanFeedbackCollector

#### Atributos:
- **feedbacks**: Lista que acumula todos os pares (response, reward)
- **current_response**: Armazena temporariamente a resposta sendo avaliada

#### Métodos:

**show_response_and_collect()**
- Exibe a resposta gerada
- Aguarda input do usuário (1/2/3/q)
- Valida entrada
- Retorna `True` se usuário digitar 'q' (quit), `False` caso contrário

**_record_feedback()**
- Converte avaliação humana em reward numérico
- Adiciona à lista de feedbacks
- Exibe emoji de confirmação (👍/😐/👎)

**get_feedback_batch()**
- Retorna todos os feedbacks coletados
- Usado para treinar o reward model

### Design Considerations

**Por que apenas 3 níveis?**
- Simplifica decisão humana (rápido e consistente)
- Reduz ambiguidade entre categorias próximas
- Suficiente para sinal de treinamento inicial

**Por que permitir 'q' (quit)?**
- Coleta de feedback é iterativa
- Usuário decide quando há dados suficientes
- Previne fadiga do avaliador

### Exemplo de Uso:
```python
collector = HumanFeedbackCollector()

# Apresenta múltiplas respostas
while True:
    response = generate_completion(model, tokenizer, prompt)
    stop = collector.show_response_and_collect(prompt, response, model_score)
    if stop:
        break

# Obtém todos os feedbacks para treinamento
batch = collector.get_feedback_batch()
```

In [6]:
class HumanFeedbackCollector:
    def __init__(self):
        self.feedbacks = []
        self.current_response = None
    
    def show_response_and_collect(self, prompt, completion, model_score):
        full_sentence = f"{prompt} {completion}"
        self.current_response = {
            'prompt': prompt,
            'completion': completion,
            'model_score': model_score
        }
        
        print(f"\n💬 Prompt: {prompt}")
        print(f"🤖 Resposta: {full_sentence}")
        print("Avalie a qualidade da resposta. Digite 1 para bom, 2 para neutro, 3 para ruim, ou q para sair.")
        
        while True:
            user_input = input("Avalie (1/2/3/q): ").strip().lower()
            if user_input == '1':
                self._record_feedback(1.0)
                return False
            elif user_input == '2':
                self._record_feedback(0.5)
                return False
            elif user_input == '3':
                self._record_feedback(0.0)
                return False
            elif user_input == 'q':
                return True
            else:
                print("Entrada inválida. Por favor, digite 1, 2, 3 ou q.")
    
    def _record_feedback(self, reward):
        self.current_response['human_reward'] = reward
        self.feedbacks.append(self.current_response.copy())
        if reward == 1.0:
            emoji = "👍"
        elif reward == 0.5:
            emoji = "😐"
        else:
            emoji = "👎"
        print(f"{emoji} Feedback registrado!")
    
    def get_feedback_batch(self):
        return self.feedbacks

feedback_collector = HumanFeedbackCollector()
print("✅ Sistema de feedback inicializado")

✅ Sistema de feedback inicializado


## 7. Treinamento do Reward Model

### Objetivo

Treinar o reward model para **prever** o feedback humano, permitindo avaliar automaticamente novas respostas sem intervenção humana constante.

### Processo em Duas Fases

#### Fase 1: Coleta de Feedback

```python
while True:
    completion = generate_completion(gen_model, gen_tokenizer, prompt)
    stop = feedback_collector.show_response_and_collect(...)
    if stop:
        break
```

**Loop interativo:**
1. Gera completion com GPT-2
2. Concatena: `"[prompt] [SEP] [completion]"`
3. Calcula score atual do reward model (para mostrar ao usuário)
4. Coleta avaliação humana
5. Repete até usuário digitar 'q'

**Por que mostrar o model_score?**
- Permite comparar predição do modelo vs feedback humano
- Ajuda identificar quando o modelo está calibrado
- Útil para debug e análise

#### Fase 2: Treinamento Supervisionado

```python
for feedback in feedbacks:
    predicted_reward = reward_model(text)
    human_reward = feedback['human_reward']
    loss = 2.0 * MSE(predicted_reward, human_reward)
    loss.backward()
```

### Loss Function: MSE (Mean Squared Error)

```python
loss = 2.0 * F.mse_loss(predicted_reward, human_reward)
```

**Componentes:**

1. **MSE**: `(predicted - actual)²`
   - Penaliza erros quadraticamente
   - Erros grandes têm penalidade muito maior
   - Incentiva predições próximas do target

2. **Fator 2.0**: Amplifica o gradiente
   - Acelera convergência (sinal de aprendizado mais forte)
   - Útil quando há poucos exemplos de treinamento
   - Pode ser ajustado empiricamente

**Exemplo de Loss:**
```
Caso 1:
predicted = 0.8, human = 1.0
MSE = (0.8 - 1.0)² = 0.04
loss = 2.0 * 0.04 = 0.08

Caso 2:
predicted = 0.3, human = 1.0
MSE = (0.3 - 1.0)² = 0.49
loss = 2.0 * 0.49 = 0.98  (penalidade muito maior!)
```

### Técnicas de Regularização

#### 1. Gradient Clipping
```python
torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
```

**Por que?**
- Previne **exploding gradients** (gradientes muito grandes)
- Limita norma L2 dos gradientes ao valor máximo de 1.0
- Estabiliza treinamento, especialmente com poucos dados

**Como funciona:**
```
Se ||gradients|| > 1.0:
    gradients = gradients / ||gradients||  # Normaliza
```

#### 2. Weight Decay (0.01)
```python
AdamW(..., weight_decay=0.01)
```

**Por que?**
- Adiciona regularização L2: `loss += 0.01 * ||weights||²`
- Previne overfitting penalizando pesos grandes
- Força o modelo a aprender representações mais simples

### Formato de Input: [SEP] Token

```python
text = f"{prompt} [SEP] {completion}"
```

**BERT espera este formato:**
- `[CLS]` é adicionado automaticamente no início
- `[SEP]` separa duas sequências (prompt e completion)
- Permite BERT processar contexto de forma estruturada

**Exemplo:**
```
Input raw: "Donald Trump is a [SEP] businessman and politician"
Tokenized: [CLS] donald trump is a [SEP] businessman and politician [SEP] [PAD]...
```

### Padding e Truncation

```python
bert_tokenizer(text, truncation=True, padding='max_length', max_length=128)
```

**Parâmetros:**
- **truncation=True**: Corta sequências longas em 128 tokens
- **padding='max_length'**: Preenche sequências curtas com [PAD]
- **max_length=128**: Tamanho fixo para batching eficiente

**Por que comprimento fixo?**
- GPUs processam batches mais eficientemente com inputs do mesmo tamanho
- 128 tokens é suficiente para prompt + completion curta

### Fluxo Completo:

```
1. Texto → Tokenização → [input_ids, attention_mask]
2. input_ids → BERT → [CLS] embedding (768 dims)
3. [CLS] → Reward Head → Score (0-1)
4. Score → MSE Loss vs Human Reward
5. Loss → Backpropagation → Atualiza pesos
6. Repete para todos os feedbacks
```

### Métricas:

**avg_loss**: Loss médio sobre todos os feedbacks
- Valores baixos (~0.01-0.1): Modelo bem calibrado
- Valores altos (~0.5+): Modelo ainda aprendendo ou dados inconsistentes

In [7]:
def collect_and_train_reward(model, prompt, gen_model, gen_tokenizer, bert_tokenizer, feedback_collector, optimizer, device):
    model.eval()
    
    print(f"\n{'='*60}")
    print(f"🎯 Coleta de Feedback")
    print(f"{'='*60}\n")
    
    while True:
        completion = generate_completion(gen_model, gen_tokenizer, prompt)
        
        text = f"{prompt} [SEP] {completion}"
        encoding = bert_tokenizer(text, truncation=True, padding='max_length', max_length=128, return_tensors='pt').to(device)
        
        with torch.no_grad():
            model_score = model(encoding['input_ids'], encoding['attention_mask']).item()
        
        stop = feedback_collector.show_response_and_collect(prompt, completion, model_score)
        if stop:
            break
    
    print(f"\n{'='*60}")
    print(f"🔥 Treinando Reward Model com Feedback")
    print(f"{'='*60}\n")
    
    feedbacks = feedback_collector.get_feedback_batch()
    
    if len(feedbacks) == 0:
        print("⚠️ Nenhum feedback coletado")
        return 0.0
    
    model.train()
    total_loss = 0.0
    
    for fb in tqdm(feedbacks, desc="Atualizando reward model"):
        text = f"{fb['prompt']} [SEP] {fb['completion']}"
        encoding = bert_tokenizer(text, truncation=True, padding='max_length', max_length=128, return_tensors='pt').to(device)
        
        optimizer.zero_grad()
        predicted_reward = model(encoding['input_ids'], encoding['attention_mask'])
        human_reward = torch.tensor([[fb['human_reward']]], dtype=torch.float32).to(device)
        loss = 2.0 * F.mse_loss(predicted_reward, human_reward)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        total_loss += loss.item()
    
    avg_loss = total_loss / len(feedbacks)
    print(f"\n✅ Loss médio: {avg_loss:.4f}")
    feedback_collector.feedbacks = []
    return avg_loss

## 8. Treinamento da Policy com Reward Model

### Reinforcement Learning para LMs

Agora que o reward model aprendeu a prever preferências humanas, usamos ele para **otimizar a policy** (GPT-2) através de reinforcement learning.

### Formulação RL

**Componentes:**
- **State**: O prompt atual
- **Action**: Escolher próximo token
- **Policy**: GPT-2 (π_θ) - distribuição de probabilidade sobre tokens
- **Reward**: Score do reward model para a sequência completa
- **Reference Policy**: GPT-2 original congelado (π_ref)

### Objetivo: Maximizar Reward com Constraint

```
maximize: E[reward(prompt, completion)]
subject to: KL(π_θ || π_ref) < threshold
```

**Tradução:**
- Queremos respostas com alto reward (alinhadas com humanos)
- MAS não queremos nos afastar muito do comportamento original
- A KL divergence mede "quão diferente" a nova policy está

### Por que KL Divergence?

**Problema sem KL penalty:**

```
Cenário:
- Reward model aprende que "exclamation marks!!!" → alto score
- Policy aprende a gerar apenas "!!!!!!!!!!!!" → reward hacking
- Resultado: Texto nonsense com alto reward artificial
```

**Solução com KL penalty:**

```python
loss = -reward + kl_coef * KL(policy || reference)
```

A policy é incentivada a:
1. Maximizar reward (primeiro termo)
2. Manter-se próxima da reference (segundo termo)

**kl_coef=0.1** controla o trade-off:
- Maior → Mais conservador (mudanças menores)
- Menor → Mais agressivo (mudanças maiores)

### Algoritmo: PPO Simplificado

Este código implementa uma versão simplificada do **PPO** (Proximal Policy Optimization), o algoritmo usado para treinar ChatGPT.

#### Loop de Treinamento:

```
Para cada step:
    Para cada sample no batch:
        1. Gera completion com policy atual
        2. Calcula reward usando reward model
        3. Calcula log-probs: policy e reference
        4. Calcula KL divergence
        5. Computa loss = -reward + kl_coef * KL
    
    6. Backward pass (soma losses do batch)
    7. Gradient clipping
    8. Update policy parameters
```

### Detalhes Técnicos

#### 1. Geração com Gradientes

```python
outputs = gen_model.generate(
    ...,
    return_dict_in_generate=True,
    output_scores=True
)
```

**Importante:**
- Normalmente `.generate()` não mantém gradientes
- `output_scores=True` retorna logits para cada token
- Permite backpropagation através da sequência gerada

#### 2. Cálculo de Log Probabilities

```python
gen_logits = gen_outputs.logits[:, :-1, :]  # Remove último token
target_ids = generated_ids[:, 1:]            # Remove primeiro token (prompt)

gen_log_probs = F.log_softmax(gen_logits, dim=-1)
gen_log_prob = gen_log_probs.gather(2, target_ids.unsqueeze(-1)).sum()
```

**O que está acontecendo:**

```
Sequência gerada: [101, 2003, 1037, 9704]
                   ↓     ↓     ↓     ↓
Logits:         logit1 logit2 logit3 logit4

Para cada posição:
- Aplicar softmax → probabilidades
- Log → log-probs
- Selecionar log-prob do token gerado
- Somar log-probs → log-prob da sequência inteira
```

**Por que log-probs?**
- Produtos de probabilidades → somas de log-probs (mais estável numericamente)
- log(p1 * p2 * p3) = log(p1) + log(p2) + log(p3)

#### 3. KL Divergence Estimation

```python
kl_penalty = gen_log_prob - ref_log_prob
```

Esta é uma aproximação do KL divergence:

```
KL(π_θ || π_ref) ≈ log π_θ(a|s) - log π_ref(a|s)
                 = log( π_θ(a|s) / π_ref(a|s) )
```

**Interpretação:**
- kl_penalty > 0: Policy dá mais probabilidade que referência (divergindo)
- kl_penalty < 0: Policy dá menos probabilidade (também divergindo)
- kl_penalty ≈ 0: Policy similar à referência

#### 4. Loss Function Final

```python
loss = -reward.squeeze() + kl_coef * kl_penalty
```

**Análise:**

**Termo 1: -reward**
- Queremos MINIMIZAR o loss
- Minimizar -reward = MAXIMIZAR reward
- Gradiente empurra policy para respostas com alto reward

**Termo 2: kl_coef * kl_penalty**
- Se policy diverge (kl_penalty alto), loss aumenta
- Gradiente empurra policy DE VOLTA para referência
- kl_coef=0.1 controla força deste "empurrão"

### Batching

```python
batch_size=4
total_loss = 0.0
for _ in range(batch_size):
    # Gera completion e calcula loss
    total_loss += loss

avg_loss = total_loss / batch_size
avg_loss.backward()
```

**Por que batch?**
- Gradientes são médias sobre múltiplas amostras
- Reduz variância do gradiente (mais estável)
- Mais eficiente computacionalmente

### Hyperparâmetros

```python
num_steps=20      # Número de iterações de otimização
batch_size=4      # Amostras por step
kl_coef=0.1       # Peso da KL penalty
learning_rate=1e-5  # Taxa de aprendizado (conservadora)
```

**Trade-offs:**
- **num_steps baixo**: Underfitting (não aprende suficiente)
- **num_steps alto**: Overfitting (memoriza feedbacks específicos)
- **kl_coef baixo**: Mais mudança, risco de reward hacking
- **kl_coef alto**: Menos mudança, pode não melhorar

### Observações Durante Treinamento

A cada 5 steps, imprime loss:
```
Step 5/20 - Loss: 0.3245
Step 10/20 - Loss: 0.2891
Step 15/20 - Loss: 0.2634
```

**Loss decrescente** indica que o modelo está:
- Gerando respostas com rewards mais altos
- OU mantendo-se mais próximo da referência
- Idealmente, ambos!

### Comparação: Supervised Learning vs RL

**Supervised Learning** (usado no reward model):
- Temos labels corretos (human rewards)
- Loss: MSE entre predição e label
- Gradiente claro e direto

**Reinforcement Learning** (usado na policy):
- Não temos labels de "resposta correta"
- Reward é esparso (só no final da sequência)
- Exploramos e aprendemos com consequências

### Conexão com ChatGPT

Este é essencialmente o algoritmo usado para treinar ChatGPT:

1. ✅ **SFT**: Fine-tune em conversas de alta qualidade (não implementado aqui)
2. ✅ **Reward Modeling**: Treina reward model com comparações humanas
3. ✅ **PPO**: Otimiza policy com reward model + KL penalty

A principal diferença é escala:
- ChatGPT: Bilhões de parâmetros, milhões de feedbacks
- Este notebook: Centenas de milhões de parâmetros, dezenas de feedbacks

In [8]:
def train_policy_with_reward(gen_model, ref_model, reward_model, tokenizer, bert_tokenizer, 
                             prompt, optimizer, device, num_steps=20, batch_size=4, kl_coef=0.1):
    """
    Treina a policy diretamente usando o reward model.
    Otimiza: reward - kl_penalty
    """
    print(f"\n{'='*60}")
    print("🔥 Treinando Policy com Reward Model")
    print(f"{'='*60}\n")
    
    gen_model.train()
    ref_model.eval()
    reward_model.eval()
    
    for step in tqdm(range(num_steps), desc="Passos de treinamento"):
        optimizer.zero_grad()
        total_loss = 0.0
        
        for _ in range(batch_size):
            # Gerar resposta
            inputs = tokenizer(prompt, return_tensors="pt").to(device)
            
            # Gerar tokens com gradientes
            outputs = gen_model.generate(
                inputs['input_ids'],
                attention_mask=inputs['attention_mask'],
                max_new_tokens=10,
                do_sample=True,
                top_p=0.95,
                temperature=0.7,
                pad_token_id=tokenizer.pad_token_id,
                return_dict_in_generate=True,
                output_scores=True
            )
            
            generated_ids = outputs.sequences
            completion_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
            completion = completion_text[len(prompt):].strip()
            
            # Calcular reward
            text = f"{prompt} [SEP] {completion}"
            encoding = bert_tokenizer(text, truncation=True, padding='max_length', 
                                     max_length=128, return_tensors='pt').to(device)
            
            with torch.no_grad():
                reward = reward_model(encoding['input_ids'], encoding['attention_mask'])
            
            # Calcular log probs da policy atual e referência
            gen_outputs = gen_model(generated_ids, attention_mask=torch.ones_like(generated_ids))
            with torch.no_grad():
                ref_outputs = ref_model(generated_ids, attention_mask=torch.ones_like(generated_ids))
            
            gen_logits = gen_outputs.logits[:, :-1, :]
            ref_logits = ref_outputs.logits[:, :-1, :]
            target_ids = generated_ids[:, 1:]
            
            gen_log_probs = F.log_softmax(gen_logits, dim=-1)
            ref_log_probs = F.log_softmax(ref_logits, dim=-1)
            
            gen_log_prob = gen_log_probs.gather(2, target_ids.unsqueeze(-1)).squeeze(-1).sum()
            ref_log_prob = ref_log_probs.gather(2, target_ids.unsqueeze(-1)).squeeze(-1).sum()
            
            # KL divergence penalty
            kl_penalty = gen_log_prob - ref_log_prob
            
            # Loss: maximizar reward - kl_penalty
            # Equivalente a minimizar: -reward + kl_coef * kl_penalty
            loss = -reward.squeeze() + kl_coef * kl_penalty
            total_loss += loss
        
        # Backprop
        avg_loss = total_loss / batch_size
        avg_loss.backward()
        torch.nn.utils.clip_grad_norm_(gen_model.parameters(), 1.0)
        optimizer.step()
        
        if (step + 1) % 5 == 0:
            print(f"\nStep {step+1}/{num_steps} - Loss: {avg_loss.item():.4f}")
    
    print("\n✅ Treinamento da policy completo!")

## 9. Execução do Pipeline RLHF

### Fluxo Completo do Treinamento

Este é o coração do notebook, onde todos os componentes anteriores se conectam:

```
┌─────────────────────────────────────────────┐
│ 1. COLETA DE FEEDBACK                       │
│    - Gera respostas com GPT-2 original      │
│    - Humano avalia qualidade (1/2/3)        │
│    - Armazena pares (response, reward)      │
└─────────────────────────────────────────────┘
                    ↓
┌─────────────────────────────────────────────┐
│ 2. TREINA REWARD MODEL                      │
│    - Aprende a prever feedback humano       │
│    - Minimiza MSE(predicted, human)         │
│    - Resultado: Modelo que avalia qualidade │
└─────────────────────────────────────────────┘
                    ↓
┌─────────────────────────────────────────────┐
│ 3. TREINA POLICY (GPT-2)                    │
│    - Usa reward model para avaliar respostas│
│    - Otimiza: maximize reward - KL penalty  │
│    - Resultado: GPT-2 alinhado com humanos  │
└─────────────────────────────────────────────┘
```

### Prompt Fixo: "Donald Trump is a"

**Por que este prompt?**

1. **Controversial**: Permite observar mudanças de comportamento claramente
2. **Open-ended**: Múltiplas completions plausíveis (businessman, politician, etc.)
3. **Teste de Alinhamento**: Mostra se o modelo aprende preferências do avaliador

**Exemplos de possíveis completions:**
- Factual: "businessman and former president"
- Neutral: "public figure who served as..."
- Negativo: "controversial figure known for..."
- Positivo: "successful entrepreneur who..."

### Interação Durante Execução

#### Fase 1: Coleta (Interativa)

```
🎯 Coleta de Feedback

💬 Prompt: Donald Trump is a
🤖 Resposta: Donald Trump is a businessman and politician from New York
Avalie a qualidade da resposta. Digite 1 para bom, 2 para neutro, 3 para ruim, ou q para sair.
Avalie (1/2/3/q): 1
👍 Feedback registrado!

💬 Prompt: Donald Trump is a
🤖 Resposta: Donald Trump is a controversial figure who has been...
Avalie (1/2/3/q): 3
👎 Feedback registrado!

[Continua até usuário digitar 'q']
```

**Recomendações:**
- Avalie pelo menos 5-10 respostas para sinal mínimo
- Seja consistente nos critérios (defina o que é "bom" antecipadamente)
- Considere: factualidade, neutralidade, completude, fluência

#### Fase 2: Treinamento do Reward (Automático)

```
🔥 Treinando Reward Model com Feedback

Atualizando reward model: 100%|████████| 10/10 [00:03<00:00,  2.89it/s]

✅ Loss médio: 0.0847
```

**O que o loss significa:**
- **< 0.1**: Excelente - modelo aprendeu bem os padrões
- **0.1 - 0.3**: Bom - modelo capturou tendências gerais
- **> 0.3**: Ruim - modelo não convergiu ou dados inconsistentes

#### Fase 3: Treinamento da Policy (Automático)

```
🔥 Treinando Policy com Reward Model

Passos de treinamento: 100%|████████| 20/20 [00:45<00:00,  2.25s/it]

Step 5/20 - Loss: 0.3421
Step 10/20 - Loss: 0.2987
Step 15/20 - Loss: 0.2654
Step 20/20 - Loss: 0.2431

✅ Treinamento da policy completo!
```

**Interpretando a progressão:**
- **Loss decrescente**: Policy está melhorando (gerando respostas com maior reward)
- **Loss crescente**: Possível overfitting ou learning rate muito alta
- **Loss oscilando**: Batch size pequeno (variância alta) ou problema de convergência

### Parâmetros Configuráveis

```python
# Reward Training
lr=2e-5              # Learning rate do reward model
weight_decay=0.01    # Regularização L2

# Policy Training
num_steps=20         # Iterações de otimização
batch_size=4         # Amostras por iteração
kl_coef=0.1         # Força da KL penalty
lr=1e-5             # Learning rate da policy
```

**Tuning sugerido:**

**Se reward model não converge:**
- Aumente lr para 5e-5
- Colete mais feedbacks (15-20)
- Reduza weight_decay para 0.001

**Se policy não melhora:**
- Aumente num_steps para 50
- Aumente batch_size para 8 (se memória permitir)
- Reduza kl_coef para 0.05 (permite mais mudança)

**Se policy se torna nonsense:**
- Reduza num_steps para 10
- Aumente kl_coef para 0.2 (mais conservador)
- Verifique se reward model está bem calibrado

### Tempo de Execução Estimado

**Coleta de Feedback:**
- Depende do usuário (1-2 min por avaliação)
- Total: ~10-20 minutos para 10 feedbacks

**Treinamento Reward Model:**
- CPU: ~30 segundos para 10 feedbacks
- GPU: ~5 segundos

**Treinamento Policy:**
- CPU: ~10-15 minutos (não recomendado)
- GPU/MPS: ~1-2 minutos

**Total: ~15-30 minutos** (a maioria é coleta de feedback)

In [9]:
fixed_prompt = "Donald Trump is a"

print("🎬 Iniciando RLHF!\n")

# 1. Coleta e treino do reward
loss = collect_and_train_reward(
    model=reward_model,
    prompt=fixed_prompt,
    gen_model=generation_model,
    gen_tokenizer=gen_tokenizer,
    bert_tokenizer=bert_tokenizer,
    feedback_collector=feedback_collector,
    optimizer=reward_optimizer,
    device=device
)

# 2. Treina policy usando reward model
train_policy_with_reward(
    gen_model=generation_model,
    ref_model=ref_model,
    reward_model=reward_model,
    tokenizer=gen_tokenizer,
    bert_tokenizer=bert_tokenizer,
    prompt=fixed_prompt,
    optimizer=policy_optimizer,
    device=device,
    num_steps=20,
    batch_size=4,
    kl_coef=0.1
)

print("\n🎉 Treinamento Completo!")

🎬 Iniciando RLHF!


🎯 Coleta de Feedback


💬 Prompt: Donald Trump is a
🤖 Resposta: Donald Trump is a bigot," Clinton spokesman Nick Merrill responded to the president
Avalie a qualidade da resposta. Digite 1 para bom, 2 para neutro, 3 para ruim, ou q para sair.
👎 Feedback registrado!

💬 Prompt: Donald Trump is a
🤖 Resposta: Donald Trump is a 'bad guy,' says White House senior advisor Ivanka
Avalie a qualidade da resposta. Digite 1 para bom, 2 para neutro, 3 para ruim, ou q para sair.
👎 Feedback registrado!

💬 Prompt: Donald Trump is a
🤖 Resposta: Donald Trump is a terrible commander-in-chief," said Rep.
Avalie a qualidade da resposta. Digite 1 para bom, 2 para neutro, 3 para ruim, ou q para sair.
👎 Feedback registrado!

💬 Prompt: Donald Trump is a
🤖 Resposta: Donald Trump is a candidate and person. His victory was an act of
Avalie a qualidade da resposta. Digite 1 para bom, 2 para neutro, 3 para ruim, ou q para sair.
👍 Feedback registrado!

💬 Prompt: Donald Trump is a
🤖 Resposta: Donal

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Atualizando reward model:   0%|          | 0/389 [00:00<?, ?it/s]


✅ Loss médio: 0.2467

🔥 Treinando Policy com Reward Model



Passos de treinamento:   0%|          | 0/20 [00:00<?, ?it/s]


Step 5/20 - Loss: -1.0304

Step 10/20 - Loss: -1.7153

Step 15/20 - Loss: -2.3878

Step 20/20 - Loss: -2.3302

✅ Treinamento da policy completo!

🎉 Treinamento Completo!


## 10. Comparação: Antes vs Depois do RLHF

### Objetivo da Comparação

Avaliar quantitativamente e qualitativamente o impacto do treinamento RLHF, comparando o comportamento do modelo original (ref_model) com o modelo treinado (generation_model).

### Metodologia

Geramos **10 frases** com cada modelo para:

1. **Avaliar consistência**: Modelos estocásticos variam entre gerações
2. **Identificar padrões**: Ver se mudanças são sistemáticas ou aleatórias
3. **Medir diversidade**: Verificar se o modelo não colapsou em uma única resposta

### Modelo de Referência (Baseline)

```python
ref_model  # Cópia congelada do GPT-2 original
```

**Características esperadas:**
- Respostas diversas refletindo distribuição do pré-treinamento
- Pode incluir vieses dos dados de treinamento originais
- Não incorpora preferências do avaliador humano
- Serve como controle experimental

**Exemplo de output típico:**
```
1. Donald Trump is a businessman and television personality
2. Donald Trump is a former president of the United States
3. Donald Trump is a controversial figure in American politics
4. Donald Trump is a real estate developer who became...
5. Donald Trump is a polarizing figure known for his...
...
```

### Modelo Treinado (RLHF)

```python
generation_model  # GPT-2 após otimização com reward model
```

**Mudanças esperadas (dependem do feedback):**

**Se feedback favoreceu neutralidade:**
- Mais frases factuais e descritivas
- Menos adjetivos carregados de valor
- Foco em informações objetivas

**Se feedback favoreceu positividade:**
- Ênfase em realizações
- Linguagem mais favorável
- Destaque para aspectos positivos

**Se feedback favoreceu crítica:**
- Menção a controvérsias
- Linguagem mais crítica
- Contexto de críticas recebidas

### Análise Qualitativa

**Aspectos a observar:**

#### 1. Tonalidade
```
Original: "Donald Trump is a controversial businessman"
Treinado: "Donald Trump is a successful entrepreneur"  (se feedback foi positivo)
```

#### 2. Completude
```
Original: "Donald Trump is a real estate"  (incompleto)
Treinado: "Donald Trump is a real estate developer and former president"  (mais completo)
```

#### 3. Factualidade
```
Original: "Donald Trump is a billionaire mogul"  (pode ser impreciso)
Treinado: "Donald Trump is a businessman who served as the 45th president"  (mais preciso)
```

#### 4. Diversidade Lexical
- Original: Usa palavras variadas
- Treinado: Pode ter vocabulário mais consistente (se reward model recompensou certos termos)

### Métricas Quantitativas (Não Implementadas, mas Úteis)

#### 1. Perplexity
```python
perplexity = exp(cross_entropy_loss)
```
- Mede quão "surpreso" o modelo fica com o texto
- Perplexity menor = modelo mais confiante
- **Esperado**: Perplexity ligeiramente maior no treinado (devido a KL penalty)

#### 2. Diversity Metrics

**Self-BLEU** (menor = mais diverso):
```python
# Calcula BLEU entre pares de respostas do mesmo modelo
# Valores baixos indicam respostas diversas
```

**Distinct-n** (maior = mais diverso):
```python
# Proporção de n-grams únicos
distinct_1 = len(unique_unigrams) / len(all_unigrams)
distinct_2 = len(unique_bigrams) / len(all_bigrams)
```

#### 3. Reward Score Médio
```python
avg_reward_original = mean([reward_model.score(s) for s in original_sentences])
avg_reward_trained = mean([reward_model.score(s) for s in trained_sentences])
```
- **Esperado**: avg_reward_trained > avg_reward_original
- Se não, o treinamento não foi efetivo

### Interpretando Resultados

#### Cenário 1: Grande Mudança
```
Original: Respostas variadas e neutras
Treinado: Respostas muito similares e consistentes
```
**Diagnóstico:**
- kl_coef muito baixo → modelo se afastou demais
- Possível reward hacking
- **Solução**: Aumentar kl_coef

#### Cenário 2: Pouca Mudança
```
Original: [respostas A, B, C]
Treinado: [respostas A', B', C']  (quase idênticas)
```
**Diagnóstico:**
- kl_coef muito alto → modelo não pôde mudar
- num_steps insuficiente
- reward model não dá sinal claro
- **Solução**: Reduzir kl_coef ou aumentar num_steps

#### Cenário 3: Mudança Desejada
```
Original: Mistura de tons e qualidades
Treinado: Consistentemente alinhado com feedback
```
**Diagnóstico:**
- ✅ RLHF funcionou como esperado
- Modelo aprendeu preferências sem colapsar

### Exemplo de Análise

**Suponha feedback favoreceu neutralidade:**

```
📊 10 Frases sem o novo treino (modelo original)
1. Donald Trump is a controversial businessman and politician
2. Donald Trump is a billionaire real estate mogul
3. Donald Trump is a former president who remains divisive
...

📊 10 Frases com o novo treino (modelo treinado)
1. Donald Trump is a businessman who served as the 45th president
2. Donald Trump is an American entrepreneur and politician
3. Donald Trump is a former president from New York
...
```

**Observações:**
- Original: Palavras como "controversial", "divisive", "mogul"
- Treinado: Palavras mais neutras "served as", "entrepreneur", "from New York"
- ✅ Modelo aprendeu a ser mais neutro conforme feedback

### Limitações desta Comparação

1. **Poucos exemplos**: 10 frases não são estatisticamente significativas
2. **Prompt único**: Não sabemos se mudanças generalizam para outros prompts
3. **Avaliação humana**: Ainda requer inspeção manual (não automatizada)
4. **Viés do avaliador**: Um único avaliador pode ter preferências idiossincráticas

### Próximos Passos

Para avaliação mais robusta:
1. Gerar 100+ frases por modelo
2. Testar em múltiplos prompts diversos
3. Avaliar com múltiplos humanos (medir concordância)
4. Calcular métricas automáticas (perplexity, diversity, reward)
5. Análise estatística (t-test para significância)

In [10]:
def generate_10_sentences(model, tokenizer, prompt):
    sentences = []
    for _ in range(10):
        completion = generate_completion(model, tokenizer, prompt)
        full_sentence = f"{prompt} {completion}"
        sentences.append(full_sentence)
    return sentences

print(f"\n{'='*60}")
print("📊 10 Frases sem o novo treino (modelo original)")
print(f"{'='*60}\n")
original_sentences = generate_10_sentences(ref_model, gen_tokenizer, fixed_prompt)
for i, sent in enumerate(original_sentences, 1):
    print(f"{i}. {sent}")

print(f"\n{'='*60}")
print("📊 10 Frases com o novo treino (modelo treinado)")
print(f"{'='*60}\n")
trained_sentences = generate_10_sentences(generation_model, gen_tokenizer, fixed_prompt)
for i, sent in enumerate(trained_sentences, 1):
    print(f"{i}. {sent}")


📊 10 Frases sem o novo treino (modelo original)

1. Donald Trump is a serial liar. He's an obstructionist." And
2. Donald Trump is a great choice for President," she said.
3. Donald Trump is a liar," the former reality TV star said, before
4. Donald Trump is a little bit of the big guy, but I think
5. Donald Trump is a white supremacist. His father, Bill Clinton, said
6. Donald Trump is a former Republican national committeeman. He is an
7. Donald Trump is a man who's been on a crusade to overturn the
8. Donald Trump is a white nationalist. We should focus on making sure Trump
9. Donald Trump is a Republican and I'm not sure what his foreign policy
10. Donald Trump is a "loser" and a "hater"

📊 10 Frases com o novo treino (modelo treinado)

1. Donald Trump is a leader


The video, which was directed
2. Donald Trump is a big success in this country," the Republican tweeted.
3. Donald Trump is a big-world winner who's already taking part in
4. Donald Trump is a much better candidate tha

## 11. Salvamento dos Modelos

### Por que Salvar Checkpoints?

Após todo o esforço de coleta de feedback e treinamento, é essencial preservar os modelos treinados:

1. **Reutilização**: Carregar modelos sem retreinar
2. **Experimentos**: Comparar diferentes configurações
3. **Deploy**: Usar modelos em produção
4. **Análise**: Investigar comportamento posteriormente

### Estrutura do Checkpoint

```python
torch.save({
    'generation_model_state_dict': generation_model.state_dict(),
    'reward_model_state_dict': reward_model.state_dict(),
    'reward_optimizer_state_dict': reward_optimizer.state_dict(),
    'policy_optimizer_state_dict': policy_optimizer.state_dict(),
}, 'rlhf_model.pt')
```

### Componentes Salvos

#### 1. `generation_model_state_dict`
**O que é:**
- Todos os pesos do GPT-2 treinado
- Parâmetros de todas as camadas (embeddings, attention, feedforward)
- Representa o modelo alinhado com feedback humano

**Tamanho:**
- ~500MB para GPT-2 base (124M parâmetros)
- Cada parâmetro: 4 bytes (float32)

**Uso posterior:**
```python
model = AutoModelForCausalLM.from_pretrained('gpt2')
checkpoint = torch.load('rlhf_model.pt')
model.load_state_dict(checkpoint['generation_model_state_dict'])
```

#### 2. `reward_model_state_dict`
**O que é:**
- Pesos do BERT + reward head
- Modelo que aprendeu a prever preferências humanas

**Por que salvar:**
- Pode ser usado para avaliar novas respostas
- Útil para debug (ver como reward mudou durante treinamento)
- Permite continuar treinamento com mais feedback

**Uso posterior:**
```python
reward_model = BERTRewardModel()
reward_model.load_state_dict(checkpoint['reward_model_state_dict'])
```

#### 3. `reward_optimizer_state_dict`
**O que é:**
- Estado interno do otimizador AdamW
- Inclui: momentos, learning rate schedule, step count

**Componentes do AdamW:**
```python
{
    'state': {
        # Para cada parâmetro:
        'step': 150,              # Número de updates
        'exp_avg': [...],         # Momento de primeira ordem (média móvel)
        'exp_avg_sq': [...]       # Momento de segunda ordem (variância móvel)
    },
    'param_groups': [{
        'lr': 2e-5,
        'weight_decay': 0.01,
        ...
    }]
}
```

**Por que salvar:**
- Continuar treinamento exatamente de onde parou
- Momentos acumulados ajudam na convergência
- Sem isso, retreinar seria mais lento

#### 4. `policy_optimizer_state_dict`
**O que é:**
- Estado do otimizador da policy (GPT-2)
- Similar ao reward optimizer

**Por que salvar:**
- Permite fine-tuning adicional
- Preserva histórico de gradientes (importante para Adam)

### Formato: PyTorch Pickle (.pt)

**Características:**
- Formato binário comprimido
- Eficiente em espaço e velocidade
- Específico do PyTorch (não portável para outros frameworks)

**Alternativas:**
```python
# Salvar apenas pesos (menor)
torch.save(model.state_dict(), 'model_weights.pt')

# Salvar modelo completo (inclui arquitetura)
torch.save(model, 'full_model.pt')

# Formato Hugging Face (portável)
model.save_pretrained('./my_model')
```

### Carregando o Checkpoint

```python
# Carregar checkpoint
checkpoint = torch.load('rlhf_model.pt', map_location=device)

# Restaurar generation model
generation_model = AutoModelForCausalLM.from_pretrained('gpt2').to(device)
generation_model.load_state_dict(checkpoint['generation_model_state_dict'])

# Restaurar reward model
reward_model = BERTRewardModel().to(device)
reward_model.load_state_dict(checkpoint['reward_model_state_dict'])

# Restaurar otimizadores (se continuar treinamento)
reward_optimizer = torch.optim.AdamW(reward_model.parameters(), lr=2e-5)
reward_optimizer.load_state_dict(checkpoint['reward_optimizer_state_dict'])

policy_optimizer = torch.optim.AdamW(generation_model.parameters(), lr=1e-5)
policy_optimizer.load_state_dict(checkpoint['policy_optimizer_state_dict'])

# Colocar em modo de avaliação
generation_model.eval()
reward_model.eval()
```

### Boas Práticas

#### 1. Versionamento
```python
# Salvar múltiplas versões
torch.save(..., f'rlhf_model_epoch_{epoch}.pt')
torch.save(..., f'rlhf_model_reward_{avg_reward:.3f}.pt')
```

#### 2. Metadata
```python
torch.save({
    'epoch': epoch,
    'timestamp': datetime.now(),
    'hyperparameters': {
        'kl_coef': 0.1,
        'num_steps': 20,
        'batch_size': 4
    },
    'metrics': {
        'reward_loss': 0.0847,
        'policy_loss': 0.2431
    },
    'generation_model_state_dict': generation_model.state_dict(),
    ...
}, 'rlhf_model.pt')
```

#### 3. Backup Regular
```python
# Durante treinamento longo
if step % 100 == 0:
    torch.save(..., f'checkpoint_step_{step}.pt')
```

### Tamanho do Arquivo

**Estimativa:**
```
GPT-2 base: ~500MB
BERT base: ~440MB
Optimizers: ~1GB (2x os pesos, pois Adam mantém momentos)
Total: ~2GB
```

**Compressão:**
```python
# Salvar com compressão
import gzip
with gzip.open('rlhf_model.pt.gz', 'wb') as f:
    torch.save(..., f)
# Reduz para ~800MB
```

### Deploy em Produção

Para usar o modelo treinado em produção:

```python
# 1. Carregar modelo
model = load_trained_model('rlhf_model.pt')
model.eval()

# 2. Inferência
with torch.no_grad():
    response = generate_completion(model, tokenizer, prompt)

# 3. (Opcional) Avaliar com reward model
score = evaluate_response(reward_model, prompt, response)
if score < threshold:
    # Regenerar ou usar fallback
    response = generate_completion(model, tokenizer, prompt)
```

### Segurança e Privacidade

**Importante:**
- Checkpoints podem conter informações sensíveis se treinados com dados privados
- Feedback humano pode revelar preferências pessoais
- **Não compartilhe publicamente** sem revisar

**Mitigação:**
- Treinar com dados públicos/sintéticos
- Anonimizar feedback
- Auditar outputs antes de release

In [ ]:
torch.save({
    'generation_model_state_dict': generation_model.state_dict(),
    'reward_model_state_dict': reward_model.state_dict(),
    'reward_optimizer_state_dict': reward_optimizer.state_dict(),
    'policy_optimizer_state_dict': policy_optimizer.state_dict(),
}, 'rlhf_model.pt')

print("✅ Modelos salvos!")

## Conclusão

### Recapitulação do Pipeline RLHF

Este notebook implementou um pipeline completo de **Reinforcement Learning from Human Feedback**, demonstrando como alinhar modelos de linguagem com preferências humanas:

```
┌──────────────────┐
│  GPT-2 Original  │  ← Modelo base pré-treinado
└────────┬─────────┘
         │
         ↓ Gera respostas
┌──────────────────┐
│ Coleta Feedback  │  ← Humano avalia: bom/neutro/ruim
└────────┬─────────┘
         │
         ↓ Treina com MSE
┌──────────────────┐
│  Reward Model    │  ← Aprende preferências (BERT)
└────────┬─────────┘
         │
         ↓ Usa como sinal
┌──────────────────┐
│  Policy Training │  ← Otimiza GPT-2 com PPO
└────────┬─────────┘
         │
         ↓
┌──────────────────┐
│ GPT-2 Alinhado   │  ← Modelo final alinhado
└──────────────────┘
```

### Conceitos-Chave Aprendidos

#### 1. Reward Modeling
- **Problema**: Não existe "resposta correta" em geração de texto
- **Solução**: Treinar modelo para imitar avaliações humanas
- **Técnica**: Supervised learning com MSE loss

#### 2. Policy Optimization
- **Objetivo**: Maximizar reward sem divergir do comportamento original
- **Técnica**: PPO simplificado com KL divergence penalty
- **Trade-off**: Reward vs. proximidade ao modelo de referência

#### 3. KL Divergence Penalty
- **Função**: Prevenir reward hacking e colapso de distribuição
- **Mecanismo**: Penalizar desvios da policy original
- **Controle**: Hyperparâmetro kl_coef ajusta conservadorismo

### Resultados Esperados

**Com feedback adequado (10+ exemplos):**
- ✅ Modelo gera respostas mais consistentes com preferências
- ✅ Loss do reward model converge (< 0.1)
- ✅ Loss da policy decresce durante treinamento
- ✅ Comparação mostra mudança no tom/conteúdo

**Sinais de sucesso:**
```python
# Antes do RLHF
avg_reward_before = 0.45  # Médio/neutro

# Depois do RLHF
avg_reward_after = 0.78   # Alinhado com preferências
```

### Limitações desta Implementação

#### 1. Escala Reduzida
- **Feedbacks**: 10-20 vs. milhões (ChatGPT)
- **Parâmetros**: 124M vs. 175B+ (GPT-3.5)
- **Prompt**: Único vs. distribuição diversa

**Impacto:**
- Generalização limitada para outros prompts
- Pode overfit nas respostas específicas avaliadas
- Não captura nuances complexas de preferências

#### 2. PPO Simplificado
**Faltando:**
- Value function (critic)
- Advantage estimation (GAE)
- Clipping de ratio de probabilidade
- Multiple epochs sobre mesmo batch

**Impacto:**
- Menos estável que PPO completo
- Pode não convergir otimamente
- Mais sensível a hyperparâmetros

#### 3. Reward Model Simples
**Limitações:**
- Treina com labels absolutos (0/0.5/1) vs. comparações pareadas
- Um único avaliador (vieses individuais)
- Não captura incerteza nas predições

**Melhor abordagem:**
- Usar comparações pareadas: "Qual resposta é melhor?"
- Múltiplos avaliadores (medir concordância)
- Modelo probabilístico (Bradley-Terry)

### Melhorias e Extensões

#### 1. Mais Dados de Feedback

**Estratégias:**
```python
# Múltiplos prompts
prompts = [
    "The climate crisis is",
    "Artificial intelligence will",
    "The best way to learn is"
]

# Coletar feedback para cada um
for prompt in prompts:
    collect_feedback(model, prompt)
```

**Benefícios:**
- Generalização entre domínios
- Reduz overfitting
- Revela padrões consistentes de preferências

#### 2. PPO Completo

**Adicionar:**
```python
class ValueModel(nn.Module):
    # Prediz reward esperado de um estado
    def forward(self, state):
        return self.value_head(self.bert(state))

# Calcular advantage
advantage = reward - value_prediction
policy_loss = -advantage * log_prob
```

**Benefícios:**
- Reduz variância do gradiente
- Convergência mais estável
- Aprende value function útil para deploy

#### 3. Comparações Pareadas

**Formato de feedback:**
```python
# Ao invés de: "Esta resposta é boa? 1/0"
# Usar: "Qual resposta é melhor? A ou B?"

response_a = generate_completion(model, prompt)
response_b = generate_completion(model, prompt)

# Usuário escolhe
preferred = ask_user_preference(response_a, response_b)

# Treinar reward model
loss = -log_sigmoid(reward(preferred) - reward(other))
```

**Benefícios:**
- Mais fácil para humanos (comparar vs. avaliar absolutamente)
- Mais consistente (elimina calibração individual)
- Usado no InstructGPT/ChatGPT

#### 4. Métricas Automáticas

**Implementar:**
```python
def evaluate_model(model, test_prompts, reward_model):
    metrics = {
        'avg_reward': [],
        'perplexity': [],
        'diversity': [],
        'toxicity': []
    }
    
    for prompt in test_prompts:
        responses = [generate_completion(model, prompt) for _ in range(10)]
        metrics['avg_reward'].append(mean([reward_model(r) for r in responses]))
        metrics['diversity'].append(calc_distinct_n(responses))
        # ...
    
    return metrics
```

#### 5. Regularização Adicional

**Técnicas:**
```python
# 1. Entropy bonus (incentiva diversidade)
entropy = -(probs * log_probs).sum()
loss = -reward + kl_penalty - entropy_coef * entropy

# 2. Length normalization (evita respostas muito curtas/longas)
reward_normalized = reward / length

# 3. Curriculum learning (começa com kl_coef alto, depois reduz)
kl_coef = initial_kl * (decay_rate ** epoch)
```

### Aplicações Práticas

#### 1. Assistentes Personalizados
```python
# Treinar modelo para estilo específico
feedbacks = collect_feedback(
    style="professional", "casual", "technical"
)
# Resultado: Assistente que adota tom preferido
```

#### 2. Content Moderation
```python
# Recompensar respostas seguras e úteis
# Penalizar conteúdo tóxico ou prejudicial
reward_model.train(safe_responses=1.0, toxic_responses=0.0)
```

#### 3. Domain Adaptation
```python
# Adaptar modelo genérico para domínio específico
# Ex: Respostas médicas, legais, técnicas
collect_domain_feedback(medical_prompts)
```

### Recursos Adicionais

**Papers Fundamentais:**
1. **InstructGPT** (OpenAI, 2022): "Training language models to follow instructions with human feedback"
2. **PPO** (Schulman et al., 2017): "Proximal Policy Optimization Algorithms"
3. **RLHF Survey** (Christiano et al., 2017): "Deep reinforcement learning from human preferences"

**Implementações:**
- **trl** (Hugging Face): Biblioteca oficial para RLHF
- **DeepSpeed-Chat**: Framework da Microsoft para RLHF em escala
- **OpenAssistant**: Projeto open-source de chatbot com RLHF

**Tutoriais:**
- Hugging Face RLHF Course: https://huggingface.co/blog/rlhf
- Spinning Up in Deep RL: https://spinningup.openai.com/

### Considerações Éticas

**RLHF não resolve todos os problemas:**
- Vieses dos avaliadores são transferidos ao modelo
- "Alinhamento" depende de quem define as preferências
- Reward hacking pode gerar comportamentos indesejados

**Boas práticas:**
1. Múltiplos avaliadores diversos
2. Auditar outputs regularmente
3. Testes de segurança (red teaming)
4. Transparência sobre limitações
5. Mecanismos de feedback contínuo

### Próximos Passos

**Para experimentar:**
1. Teste com diferentes prompts (diversos domínios)
2. Ajuste hyperparâmetros (kl_coef, learning rates)
3. Compare com baselines (sem RLHF)
4. Implemente métricas automáticas
5. Escale para mais feedback (50-100 exemplos)

**Para produção:**
1. Implemente PPO completo
2. Use comparações pareadas
3. Adicione value function
4. Escale para modelos maiores (GPT-2 Large, etc.)
5. Deploy com monitoramento contínuo

### Conclusão Final

RLHF é uma técnica poderosa para alinhar modelos de linguagem com valores humanos. Este notebook demonstra os princípios fundamentais em escala reduzida, mas os conceitos escalam para sistemas como ChatGPT.

A chave é entender o trade-off entre **exploração** (tentar comportamentos novos) e **conservadorismo** (manter comportamento seguro), controlado pela KL penalty.

**Mensagem final:**
> "Modelos de linguagem são ferramentas poderosas. RLHF nos permite guiá-las para serem úteis, inofensivas e honestas. Mas o alinhamento é um processo contínuo, não um destino final."

**🎉 Parabéns por completar este tutorial de RLHF!**